In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from keras.layers import Dense, Flatten
from keras.utils import to_categorical
from keras.models import Sequential
from keras.datasets import mnist

In [ ]:
################################################################################################
####################################### DATASET CSV LOAD #######################################

train_data_size   = 405131
test_data_size    = 67732
height_pixel_size = 28
width_pixel_size  = 28

# Loads train image data and verify with initial data
loaded_df_train_image_data = pd.read_csv('dataset\\train_image_data.csv')
loaded_array_train_image_data = loaded_df_train_image_data.to_numpy().reshape(train_data_size, height_pixel_size, width_pixel_size)

# Loads train label data and verify with initial data
loaded_df_train_label_data = pd.read_csv('dataset\\train_label_data.csv')
loaded_array_train_label_data = loaded_df_train_label_data.to_numpy().reshape(train_data_size)

# Loads test image data and verify with initial data
loaded_df_test_image_data = pd.read_csv('dataset\\test_image_data.csv')
loaded_array_test_image_data = loaded_df_test_image_data.to_numpy().reshape(test_data_size, height_pixel_size, width_pixel_size)

# Loads test label data and verify with initial data
loaded_df_test_label_data = pd.read_csv('dataset\\test_label_data.csv')
loaded_array_test_label_data = loaded_df_test_label_data.to_numpy().reshape(test_data_size)

In [ ]:
# Set train and test data names to x and y for simplicity
x_train, y_train = loaded_array_train_image_data, loaded_array_train_label_data
x_test, y_test = loaded_array_test_image_data, loaded_array_test_label_data

In [ ]:
# Normalization
x_train = x_train / 255.0
x_test = x_test / 255.0

# Set outputs as categorical. eg. 4 to [0,0,0,0,1,0,0,0...]
# There are 26 classes total (symbols, characters etc.)
y_train = to_categorical(y_train, 26)
y_test = to_categorical(y_test, 26)

# Create model
model = Sequential([
  Flatten(input_shape=(28, 28)),
  Dense(128, activation='relu'),
  Dense(26, activation='softmax')
])

# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(x_train, y_train, epochs=8, validation_data=(x_test, y_test))

In [ ]:
# Save model
model.save("models\channo_v0.1.keras")

In [ ]:
# Plot Loss-Epoch Graph
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.legend()
plt.show()

# Plot Accuracy-Epoch Graph 
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
################################################################################################
###################################### CUSTOM IMAGE TEST #######################################

# Load model
model = tf.keras.models.load_model("models\channo_v0.1.keras")

# Load image
img_path = 'images\\chessmove_7_filtered_1.png'
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# Arrange the image available for feeding to the model
img = cv2.resize(img, (28, 28), interpolation = cv2.INTER_AREA)
img = 255 - img
# img[img < 120] = 0
img = img / 255.0
img = img.reshape(1, 28, 28)

# Show image
plt.imshow(img[0], cmap='gray')
plt.show()

# Pass image to our model
predictions = model.predict(img)
print(predictions[0])
predicted_class = np.argmax(predictions[0])

print('Predicted class: ', predicted_class)